In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import cross_val_score, StratifiedKFold

from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

### Load the data

In [2]:
def load_data(dataset):
    train_url = os.path.join('../data/playground_series_s4e1', 
                         'train.csv')
    test_url = os.path.join('../data/playground_series_s4e1', 
                            'test.csv')
    origin_url = os.path.join('../data/playground_series_s4e1', 
                              'Churn_Modelling.csv')
    if dataset == 'train':
        df = pd.read_csv(train_url)
    elif dataset == 'test':
        df = pd.read_csv(test_url)
    elif dataset == 'origin':
        df = pd.read_csv(origin_url)
    else:
        raise ValueError(f'{dataset} is not a supported dataset')
    
    return df

### Target Imputation

In [3]:
def impute_target(df, col):
    # calcualted the mean exited rate by specified columns
    df_target = df.groupby(col).agg({'exited': 'mean'})
    df_target = df_target.reset_index()
    df_target = df_target.rename(columns={'exited': col+'_target'})
    
    df = pd.merge(df, df_target, on=col, how='left', 
                  validate='m:1')
    df = df.drop(columns=[col])
    
    return df

### Baseline Models
Accuracy: roc_auc
1. logistic regression
2. catboost classifier
3. xgboost classifier
4. lightgbm classifier

In [4]:
def calculate_score(df, model='lr', scaler='minmax'):
    
    X_train = df.drop(columns=['exited'])
    y_train = df['exited']
    
    # stadardization
    if scaler == 'minmax':
        scaler = MinMaxScaler()
    elif scaler == 'standard':
        scaler = StandardScaler()
    else:
        raise ValueError(f'{scaler} is not supported')
    X_train_scaled = scaler.fit_transform(X_train)
    
    # cross validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    if model == 'lr':
        model = LogisticRegression()
    elif model == 'cat':
        model = CatBoostClassifier(verbose=0)
    elif model == 'xgb':
        model = XGBClassifier(verbosity=0)
    elif model == 'lgb': 
        model = LGBMClassifier(verbose=-1)
    else:
        model = model
    scores = cross_val_score(model, X_train_scaled, y_train, cv=skf, scoring='roc_auc')
    
    return scores

In [5]:
%%time
# load the data
train_df = load_data('train')
train_df.columns = train_df.columns.str.lower()
train_df = train_df.drop(columns=['id', 'customerid'])

# impute the dataset
cols = ['surname', 'geography', 'gender', 'hascrcard', 'isactivemember']
for col in cols:
    train_df = impute_target(train_df, col)
    
# calculate the roc scores
for model in ['lr', 'cat', 'xgb', 'lgb']:
    for scaler in ['minmax', 'standard']: 
        scores = calculate_score(train_df, model=model, scaler=scaler)
        print(f'The average score of {model} with {scaler} is {scores.mean(): .4f}.')

The average score of lr with minmax is  0.8353.
The average score of lr with standard is  0.8353.
The average score of cat with minmax is  0.8996.
The average score of cat with standard is  0.8995.
The average score of xgb with minmax is  0.8981.
The average score of xgb with standard is  0.8982.
The average score of lgb with minmax is  0.9000.
The average score of lgb with standard is  0.8998.
CPU times: total: 32min 23s
Wall time: 2min 42s


### Hyperparameter Tuning

In [6]:
import optuna
from sklearn.metrics import roc_auc_score
from optuna.integration import LightGBMPruningCallback

In [20]:
def objective(trial, X, y):
    
    param_grid = {
        
        # tree structure
        'max_depth': trial.suggest_int('max_depth', 3, 12, step=1),
        'max_leaves': trial.suggest_int('max_leaves', 20, 3000, step=20), 

        # better accuracy
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.9, step=0.01),
        'n_estimators': trial.suggest_categorical('n_estimators', [10000]),

        # combat overfitting
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.2, 0.99, log=True),
        'subsample': trial.suggest_float('subsample', 0.2, 0.99, log=True),
        'subsample_freq': trial.suggest_categorical('subsample_freq', [1]), 
        'reg_alpha': trial.suggest_categorical('reg_alpha', [0.1, 0.5, 1.0, 5.0, 10.0, 50.0, 100.0]), # L1 regularization
        'reg_lambda': trial.suggest_categorical('reg_lambda', [0.1, 0.5, 1.0, 5.0, 10.0, 50.0, 100.0]), # L2 regularization

        'random_state': trial.suggest_categorical('random_state', [42]), 
        'n_jobs': trial.suggest_categorical('n_jobs', [-1]), 
        'early_stopping_rounds': trial.suggest_categorical('early_stopping_rounds', [100]), 
        'eval_metric': trial.suggest_categorical('eval_metric', ['auc']), 
    }
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    cv_scores = np.empty(5)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        model = XGBClassifier(objective='binary:logistic', **param_grid)
        model.fit(
            X_train, 
            y_train, 
            eval_set=[(X_test, y_test)], 
            #eval_metric='auc', 
            #early_stopping_rounds=100,
            #callbacks=[LightGBMPruningCallback(trial, 'auc'), ], 
            verbose=0, 
        )
        y_preds = model.predict_proba(X_test)[:, 1]
        cv_scores[idx] = roc_auc_score(y_test, y_preds)
    
    return np.mean(cv_scores)

In [21]:
%%time
X_train = train_df.drop(columns=['exited'])
y_train = train_df['exited']

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)

study = optuna.create_study(direction='maximize', study_name='XGB Classifier')
func = lambda trial: objective(trial, X_train_scaled, y_train)
study.optimize(func, n_trials=20)

[I 2024-01-04 14:28:01,090] A new study created in memory with name: XGB Classifier
[I 2024-01-04 14:29:19,867] Trial 0 finished with value: 0.8971765313887399 and parameters: {'max_depth': 12, 'max_leaves': 680, 'learning_rate': 0.86, 'n_estimators': 10000, 'colsample_bytree': 0.7755750129863637, 'subsample': 0.3133886566213953, 'subsample_freq': 1, 'reg_alpha': 100.0, 'reg_lambda': 0.1, 'random_state': 42, 'n_jobs': -1, 'early_stopping_rounds': 100, 'eval_metric': 'auc'}. Best is trial 0 with value: 0.8971765313887399.
[I 2024-01-04 14:29:36,685] Trial 1 finished with value: 0.8971499114327066 and parameters: {'max_depth': 4, 'max_leaves': 1300, 'learning_rate': 0.54, 'n_estimators': 10000, 'colsample_bytree': 0.4224457325155376, 'subsample': 0.21665061696578028, 'subsample_freq': 1, 'reg_alpha': 0.1, 'reg_lambda': 10.0, 'random_state': 42, 'n_jobs': -1, 'early_stopping_rounds': 100, 'eval_metric': 'auc'}. Best is trial 0 with value: 0.8971765313887399.
[I 2024-01-04 14:31:02,134] Tr

KeyboardInterrupt: 

In [18]:
print(f'\tBest value： {study.best_value:.5f}')

	Best value： 0.89755


In [22]:
model = XGBClassifier(objective='binary:logistic', **study.best_params)
scores = calculate_score(train_df, model)
scores.mean()

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Fan\Anaconda3\envs\gradientboosting\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Fan\Anaconda3\envs\gradientboosting\lib\site-packages\xgboost\core.py", line 575, in inner_f
    return f(**kwargs)
  File "C:\Users\Fan\Anaconda3\envs\gradientboosting\lib\site-packages\xgboost\sklearn.py", line 1400, in fit
    self._Booster = train(
  File "C:\Users\Fan\Anaconda3\envs\gradientboosting\lib\site-packages\xgboost\core.py", line 575, in inner_f
    return f(**kwargs)
  File "C:\Users\Fan\Anaconda3\envs\gradientboosting\lib